### 必要なライブラリのインストール

In [24]:
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import requests

### URLを指定しスクレイピング

In [25]:
res = requests.get('https://www.tourism.jp/tourism-database/stats/outbound/#monthly-ports')
soup = BeautifulSoup(res.text, 'html.parser')

In [26]:
# 空港名取得
airport_tags = soup.find_all("th", {"rowspan": "2"})
airports = [tag.text for tag in airport_tags]

# 値取得
value_tags = soup.find('table', {"class": "table_airports"}).find_all('td')
# 値のリスト作成
values = [tag.text for tag in value_tags]
# 空港ごとの値に分割
values_per_airport = [values[i::len(airports)] for i in range(len(airports))]

# 各空港に12個の値が紐づいていることを前提に、値を空港ごとに分割
num_values_per_airport = 12  # 各空港の値の数
values_per_airport = [values[i:i + num_values_per_airport] 
                      for i in range(0, len(values), num_values_per_airport)]

### データフレーム作成とデータベースへ保存

In [27]:
# DataFrame作成
# values_per_airportの転置を行う前に、行と列の数を一致させる
df = pd.DataFrame(values_per_airport, columns=[f'{i}' for i in range(num_values_per_airport)]).T  
# 列名を空港名に設定
df.columns = airports[:len(df.columns)]  # 列数が空港数を超えないように調整

# SQLiteに保存
conn = sqlite3.connect('scrapped_data.db')
df.to_sql('airports', conn, if_exists='replace', index_label='month')
conn.close()

### データベースへ接続

In [28]:
# データベースに接続
conn = sqlite3.connect('scrapped_data.db')

# 'airports'テーブルからデータを取得
df = pd.read_sql_query("SELECT * from airports", conn)
df['month'] = ['6', '7', '8', '9', '10', '11', '6', '7', '8', '9', '10', '11']


# 結果を表示
print(df)

# データベース接続を閉じる
conn.close()

   month  新千歳CTS 東京/成田NRT 東京/羽田HND   中部NGO    関西KIX   福岡FUK 博多港HKP   那覇OKA
0      6   6,572  264,916  330,408  56,592  171,879  52,616  3,802   5,662
1      7   4,074  323,826  370,668  62,327  193,174  53,953  1,463   5,481
2      8   7,587  466,522  458,222  87,662  276,060  82,196  2,853   7,657
3      9  10,307  376,694  405,953  69,382  228,387  65,954  1,065   7,576
4     10  12,188  340,575  377,669  74,126  215,352  68,877  1,717   7,974
5     11  13,421  346,189  373,809  79,456  223,427  71,058  1,882  10,546
6      6    +68%    -2.9%    +0.6%  +26.5%   +11.2%   +7.8%    +0%     +0%
7      7  +15.4%   -16.7%    -9.2%   +1.1%    -9.9%  -25.2%    +0%     +0%
8      8  +23.3%   +49.6%   +26.7%    +73%   +55.6%  +45.6%    +0%     +0%
9      9  +35.8%   +37.4%   +21.1%  +27.3%   +40.1%  +20.8%    +0%     +0%
10    10  +59.1%   +11.8%    +6.8%  +16.6%   +16.4%  +10.8%    +0%     +0%
11    11  +30.5%   +17.4%   +18.2%  +35.2%   +26.2%  +34.7%    +0%     +0%
